文章在这个地方 http://online.cambridgecoding.com/notebooks/eWReNYcAfB/implementing-your-own-recommender-systems-in-python-2

In [1]:
import numpy as np
import pandas as pd

In [2]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('~/Downloads/ml-100k/u.data', sep='\t', names=header)

IOError: File ml-100k/u.data does not exist

In [ ]:
n_users = df.user_id.unique().size
n_items = df.item_id.unique().size
print 'users = {}, items = {}'.format(n_users, n_items)

In [ ]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.25)

In [ ]:
train_mx = np.zeros((n_users, n_items))
test_mx = np.zeros((n_users, n_items))

In [ ]:
# iterrows可以用来遍历df
for idx, row in train_data.iterrows():
    train_mx[row['user_id']-1][row['item_id'] -1] = row['rating']
for idx, row in test_data.iterrows():
    test_mx[row['user_id']-1][row['item_id'] -1] = row['rating']

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
# 这个地方比较奇怪，是因为如果使用pairwise_distances输入两个相同向量的话，结果为0
# 而这个地方实际上我们希望如果相近的话，数值越高才对
user_sim = 1 - pairwise_distances(train_mx, metric='cosine')
item_sim = 1- pairwise_distances(train_mx.T, metric = 'cosine')

下面这个函数有几个地方挺有意思的

np.newaxis 可以用来把多开辟一个维度. 比如原来是(10,)可以扩展为(10, 1).

np.dot 这个函数如果用于矩阵上的话，效果应该是这样的
```
a = [0,1,2
3,4,5
6,7,8]
```
如果a.dot(a)的话，那么第一行元素值应该是
```
0 * [0,1,2] + 1 * [3,4,5] + 2 * [6,7,8] = [15, 18, 21]
```
效果是加权求和

**这个地方搞糊涂了. np.dot就是矩阵相乘. np.multiply点乘**

下面函数里面b的这个地方。如果我们希望比如
```
a = [0,1,2
3,4,5
6,7,8]
b = [3, 4, 5]
```
结果是
```
[0 / 3, 1 / 3, 2 / 3,
 3 / 4, 4 / 4, 5 / 4,
 6 / 5, 7 / 5, 8 / 5]
```
的话，那么可以使用这段代码. `b2 = np.array([b]); a / b2.T`

In [ ]:
def user_pred(mx, sim):
    ratings_mean = mx.mean(axis = 1)
    ratings_mean2 = ratings_mean[:, np.newaxis]
    print mx.shape, ratings_mean2.shape
    ratings_diff = mx - ratings_mean2
    a = sim.dot(ratings_diff)
    b = np.abs(sim).sum(axis = 1)[:, np.newaxis]
    b2 = np.array([np.abs(sim).sum(axis=1)]).T
    print a.shape, b.shape, b2.shape
    assert(np.all(b == b2))
    base_pred = a / b
    pred = base_pred + ratings_mean2
    return pred

In [ ]:
user_prediction = user_pred(train_mx, user_sim)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
# 从里面筛选出有ratings的部分. nonzeros()返回的是两个数组. index的row, col.
# print test_mx.nonzero()
result = user_prediction[test_mx.nonzero()]
truth = test_mx[test_mx.nonzero()]
np.sqrt(mean_squared_error(result, truth))

In [ ]:
def item_pred(mx, sim):
    a = mx.dot(sim)
    b = np.array([np.abs(sim + 0.1).sum(axis=1)])
    pred = a / b
    return pred

In [ ]:
item_prediction = item_pred(train_mx, item_sim)
result = item_prediction[test_mx.nonzero()]
truth = test_mx[test_mx.nonzero()]
np.sqrt(mean_squared_error(result, truth))

In [ ]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)

In [ ]:
import scipy.sparse as sp
# 注意svds在scipy里面
from scipy.sparse.linalg import svds

In [ ]:
u,s,v = svds(train_mx, k = 20) # 分解出20个latent factors.
s_diag = np.diag(s)

In [ ]:
x_pred = np.dot(np.dot(u, s_diag), v)
result = x_pred[test_mx.nonzero()]
np.sqrt(mean_squared_error(result, truth))